<p style='text-align: right;'><span style="color: #000508; font-family: Segoe UI; font-size: 0.8em; font-weight: 300;">Image source: www.freepik.com</span></p>

![](https://i.ibb.co/fXHNQtH/cover1-01.jpg)

<p style='text-align: center;'><span style="color: #000508; font-family: Segoe UI; font-size: 2.5em; font-weight: 300;">RSNA 3D CLAHE Voxels + TPU 3D Augmentations</span></p>
<br>

<span style="color: #1e009c; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">Overview</span>
<br>
<br>
&ensp;✔️&ensp;Align, Crop & Filter Voxels<br>
&ensp;✔️&ensp;Normalization & Contrast Enhancement Across the 3D Volume<br>
&ensp;✔️&ensp;Resample Voxels to size 64 x 256 x 256 for Optimal Training<br>
&ensp;✔️&ensp;Efficient 3D Augmentations for TPU & GPU<br>
&emsp;&emsp;&emsp;✔️&ensp;Flip<br>
&emsp;&emsp;&emsp;✔️&ensp;Gamma<br>
&emsp;&emsp;&emsp;✔️&ensp;Brightness<br>
&emsp;&emsp;&emsp;✔️&ensp;Contrast<br>
&emsp;&emsp;&emsp;✔️&ensp;Cutout<br>
&emsp;&emsp;&emsp;✔️&ensp;Random Resized Crop<br>
&emsp;&emsp;&emsp;✔️&ensp;Rotate<br>
&emsp;&emsp;&emsp;✔️&ensp;Blur<br>
&ensp;✔️&ensp;Interactive Visualizations with Weights & Biases<br>

<br>

We resize and sample the competetion dataset to 64x256x256 (in the current notebook version). This size seems to be an arbitrary size to train on TPUs with a large batch size and to train larger model architectures as well without OOM issues.
<br>
<br>

The augmentations are carefully applied so the transformed image characteristics are uniform throughout the voxel, also preserving the geometrical alignment between slices to minimize discontinuities.
<br>
<br>

Prior to augmentations, we align planes as the MRI plane type (Axial, Coronal, and Sagittal) is not consistent among patients or MRI scan types (FLAIR, T1w, T1wCE, T2w) and since it is better to train models using MRI voxels that are consistent in terms of plane type. 
<br>
<br>

Transpose and rotate operations are used to appropriately align the scans across planes. The scans are then normalized and cropped to remove excess space around the ROI. Some frames with less information are also filtered out. Then, we resize and resample the scans to the size 64 x 256 x 256 and we make sure to keep the plane type consistant (axial) across all the scans.
<br>
<br>

Contrast enhancement is done across the 3D volume with CLAHE.

<br>
<br>
<br>

<p style='text-align: left;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.1em; font-weight: 600;"> 🏷️ TPU 3D-CNN Training Notebook</span></p>

&emsp;&emsp;[[TPU] RSNA Keras 3D CNN Voxel Train 🏷️🏂](https://www.kaggle.com/sreevishnudamodaran/tpu-rsna-keras-3d-cnn-voxel-train)<br>

<p style='text-align: left;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.1em; font-weight: 600;"> 🏷️ Dataset with Processed 64 x 256 x 256 Voxels</span></p>

&emsp;&emsp;[RSNA Processed Voxels 64x256x256](https://www.kaggle.com/sreevishnudamodaran/rsna-processed-voxels-64x256x256)

<p style='text-align: left;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.1em; font-weight: 600;"> 🏷️ Dataset with Processed 64 x 256 x 256 Voxels with CLAHE</span></p>

&emsp;&emsp;[RSNA Processed Voxels 64x256x256 CLAHE](https://www.kaggle.com/sreevishnudamodaran/rsna-processed-voxels-64x256x256-clahe)

<br>
<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">References:</span>
<p style='text-align: left;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.1em; font-weight: 600;"> - Thanks to <a href="https://www.kaggle.com/ren4yu">@ren4yu</a> for the notebook <a href="https://www.kaggle.com/ren4yu/normalized-voxels-align-planes-and-crop">Normalized Voxels: Align Planes and Crop</a>, which details the aligning, normalizing, croping of voxels</span></p>

<p style='text-align: left;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.1em; font-weight: 600;"> - <a href="https://github.com/VincentStimper/mclahe">Multidimensional Contrast Limited Adaptive Histogram Equalization (MCLAHE)</a></span></p>

<p style='text-align: left;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.1em; font-weight: 600;"> - GIF visualizations are adapted from the <a href="https://www.kaggle.com/ayuraj/brain-tumor-eda-and-interactive-viz-with-w-b">Brain Tumor EDA and Interactive Viz with W&B</a> by <a href="https://www.kaggle.com/ayuraj">@ayuraj</a></span></p>


<br>
<br>
<span style="float:center;"><a href="https://www.kaggle.com/sreevishnudamodaran"><img style="padding: 5px;" border="0" alt="Ask Me Something" src="https://img.shields.io/badge/Ask%20me-something-7a43bc.svg?style=for-the-badge&logo=kaggle" width="160" height="20"></a><br>
    <img style="padding: 5px;" border="0" alt="Ask Me Something" src="https://img.shields.io/badge/Please-Upvote%20If%20you%20like%20this-16a5e9?style=for-the-badge&logo=kaggle" width="250" height="20"></span>
<br>

In [ ]:
!pip install --upgrade https://github.com/VincentStimper/mclahe/archive/numpy.zip -q

In [ ]:
import os
import math
from pathlib import Path
import glob
import math

import numpy as np
import pandas as pd
import cv2
import pydicom
import mclahe as mc

from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import rcParams
from matplotlib.animation import FuncAnimation
matplotlib.rcParams['animation.html'] = 'jshtml'

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from IPython.display import IFrame
from IPython.core.display import display, HTML
import imageio

<span style="color: #1e009c; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">Process & Create Voxels</span>

In [ ]:
# https://www.kaggle.com/ren4yu/normalized-voxels-align-planes-and-crop
# https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data
# https://www.kaggle.com/davidbroberts/determining-mr-image-planes
def get_image_plane(data):
    x1, y1, _, x2, y2, _ = [round(j) for j in data.ImageOrientationPatient]
    cords = [x1, y1, x2, y2]

    if cords == [1, 0, 0, 0]:
        return 'Coronal'
    elif cords == [1, 0, 0, 1]:
        return 'Axial'
    elif cords == [0, 1, 0, 0]:
        return 'Sagittal'
    else:
        return 'Unknown'
    
def get_voxel(study_id, scan_type):
    imgs = []
    dcm_dir = data_root.joinpath(DATASET, study_id, scan_type)
    dcm_paths = sorted(dcm_dir.glob("*.dcm"), key=lambda x: int(x.stem.split("-")[-1]))
    positions = []
    
    for dcm_path in dcm_paths:
        img = pydicom.dcmread(str(dcm_path))
        imgs.append(img.pixel_array)
        positions.append(img.ImagePositionPatient)
        
    plane = get_image_plane(img)
    voxel = np.stack(imgs)
    
    # reorder planes if needed and rotate voxel
    if plane == "Coronal":
        if positions[0][1] < positions[-1][1]:
            voxel = voxel[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        voxel = voxel.transpose((1, 0, 2))
    elif plane == "Sagittal":
        if positions[0][0] < positions[-1][0]:
            voxel = voxel[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        voxel = voxel.transpose((1, 2, 0))
        voxel = np.rot90(voxel, 2, axes=(1, 2))
    elif plane == "Axial":
        if positions[0][2] > positions[-1][2]:
            voxel = voxel[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        voxel = np.rot90(voxel, 2)
    else:
        raise ValueError(f"Unknown plane {plane}")
    return voxel, plane

def normalize_contrast(voxel):
    if voxel.sum() == 0:
        return voxel
    voxel = voxel - np.min(voxel)
    voxel = voxel / np.max(voxel)
    voxel = (voxel * 255).astype(np.uint8)
    return voxel

def crop_voxel(voxel):
    if voxel.sum() == 0:
        return voxel
    keep = (voxel.mean(axis=(0, 1)) > 0)
    voxel = voxel[:, :, keep]
    keep = (voxel.mean(axis=(0, 2)) > 0)
    voxel = voxel[:, keep, :]
    keep = (voxel.mean(axis=(1, 2)) > 0)
    voxel = voxel[keep, :, :]
    return voxel

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">Re-sample & Filter Voxels</span>

Some of the leading and the trailing images from each voxel are removed by applying a filter based on the standard deviation of the voxel mean values.

We also resample the voxels to the same axial MRI plane orientation throughout the data.

Sampling planes is done along the longest axis while resizing to reduce degradation.

In [ ]:
def filter_voxel(voxel, filter_thr):
    voxel_mean = voxel.mean(axis=(1, 2))
    keep = (voxel_mean > voxel_mean.std()*filter_thr)
    voxel = voxel[keep, :, :]
    return voxel

def resize_voxel(voxel, sz=(64, 256, 256)):
    output = np.zeros((sz[0], sz[1], sz[2]), dtype=np.uint8)
    if np.argmax(voxel.shape) == 0:
        for i, s in enumerate(np.linspace(0, voxel.shape[0] - 1, num=sz[0])):
            sampled = voxel[int(s), :, :]
            output[i, :, :] = cv2.resize(sampled, (sz[2], sz[1]), cv2.INTER_CUBIC)
    elif np.argmax(voxel.shape) == 1:
        for i, s in enumerate(np.linspace(0, voxel.shape[1] - 1, num=sz[1])):
            sampled = voxel[:, int(s), :]
            output[:, i, :] = cv2.resize(sampled, (sz[2], sz[0]), cv2.INTER_CUBIC)
    elif np.argmax(voxel.shape) == 2:
        for i, s in enumerate(np.linspace(0, voxel.shape[2] - 1, num=sz[2])):
            sampled = voxel[:, :, int(s)]
            output[:, :, i] = cv2.resize(sampled, (sz[1], sz[0]), cv2.INTER_CUBIC)
    return output

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">3D CLAHE</span>

We use the Multidimensional Contrast Limited Adaptive Histogram Equalization (MCLAHE) implementation for contrast enhancement across arbitrary number of dimensions, to apply CLAHE across our 3D volume.

Repo: https://github.com/VincentStimper/mclahe

V. Stimper, S. Bauer, R. Ernstorfer, B. Schölkopf and R. P. Xian, "Multidimensional Contrast Limited Adaptive Histogram Equalization," in IEEE Access, vol. 7, pp. 165437-165447, 2019.

We perform the MCLAHE operation on GPU with TF for a slight performance boost.

In [ ]:
def clahe_3d(voxel):
    voxel = mc.mclahe(voxel, kernel_size=[8,32,32],
              n_bins=128,
              clip_limit=0.01,
              adaptive_hist_range=False)
    return (voxel*255.).astype(np.uint8).clip(0, 255)

@tf.function(input_signature=[tf.TensorSpec(None, tf.uint8)])
def tf_clahe_3d(voxel):
    voxel = tf.numpy_function(clahe_3d, [voxel], tf.uint8)
    return voxel

In [ ]:
DATASET = 'train'
scan_types = ['FLAIR','T1w','T1wCE','T2w']
data_root = Path("../input/rsna-miccai-brain-tumor-radiogenomic-classification")

voxel_dir = '/kaggle/working/voxels'
os.makedirs(voxel_dir, exist_ok=True)

filter_thr = 0.4 # voxel_mean > voxel_std_deviation*filter_thr

avgs = []

for study_path in tqdm(list(data_root.joinpath(DATASET).glob("*"))):
    study_id = study_path.name
        
    if study_id in ['00109', '00123', '00709']:
        continue

    for i, scan_type in enumerate(scan_types):
        voxel, plane = get_voxel(study_id, scan_type)
        voxel = normalize_contrast(voxel)
        voxel = crop_voxel(voxel)
        voxel = filter_voxel(voxel, filter_thr)
        voxel = resize_voxel(voxel, sz=(64, 256, 256))
        
        voxel = tf_clahe_3d(voxel)
        
        os.makedirs(f'{voxel_dir}/{scan_type}', exist_ok=True)
        
        with open(f'{voxel_dir}/{scan_type}/{study_id}.npy', 'wb') as f:
#             np.save(f, voxel)
            np.save(f, voxel.numpy())

In [ ]:
# !zip -rq rsna-voxels-64x256x256-aligned-clahe.zip $voxel_dir/*

In [ ]:
# from kaggle_secrets import UserSecretsClient
# from google.cloud import storage

# def upload_blob(bucket_name, source_file_name, destination_blob_name):
#     """Uploads a file to the bucket. https://cloud.google.com/storage/docs/ """
#     bucket = storage_client.get_bucket(bucket_name)
#     blob = bucket.blob(destination_blob_name)
#     blob.upload_from_filename(source_file_name)
#     print('File {} uploaded to {}.'.format(
#         source_file_name,
#         destination_blob_name))

# user_secrets = UserSecretsClient()
# gcp_project_id = user_secrets.get_secret("gcp_project_id")
# bucket_name = user_secrets.get_secret("bucket_name")
# storage_client = storage.Client(project=gcp_project_id)
# upload_blob(bucket_name, 'rsna-voxels-64x256x256-aligned-clahe.zip', 'rsna-voxels-64x256x256-aligned-clahe.zip')

In [ ]:
# !rm -r $voxel_dir ./rsna-voxels-64x256x256-aligned-clahe.zip

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">Visualize Samples</span>

The generated datasets are here:

&emsp;[RSNA Processed Voxels 64x256x256](https://www.kaggle.com/sreevishnudamodaran/rsna-processed-voxels-64x256x256)<br>

&emsp;[RSNA Processed Voxels 64x256x256 CLAHE](https://www.kaggle.com/sreevishnudamodaran/rsna-processed-voxels-64x256x256-clahe)

Let's read it directly.

In [ ]:
def show_animation(images, normalized=True):
    fig = plt.figure(figsize=(4, 4))
    plt.axis('off')
    if normalized:
        im = plt.imshow(images[0], cmap='gray', vmin=0.0, vmax=1.0)
    else:
        im = plt.imshow(images[0], cmap='gray', vmin=0.0, vmax=255.0)
    def animate(i):
        im.set_array(images[i])
        # return the artists set
        return [im]
    display(FuncAnimation(fig, animate, frames=len(images),
                                               interval=20))
    plt.close()

voxels = np.load('../input/rsna-processed-voxels-64x256x256-clahe/voxels/FLAIR/00046.npy')
show_animation([voxel/255. for voxel in voxels])

In [ ]:
voxels = np.load('../input/rsna-processed-voxels-64x256x256-clahe/voxels/FLAIR/00046.npy')

sqrt = math.ceil(math.sqrt(voxels.shape[0]))
fig = plt.figure(figsize=(11, 11))
fig.suptitle('Axial Plane', x=0.5, y=0.98, size=25)

for idx in range(voxels.shape[0]):
    ax = fig.add_subplot(int(sqrt), int(sqrt), idx+1)
    ax.imshow(voxels[idx, : , :], cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
fig.tight_layout()
plt.savefig(f'samples.png')
fig.show()

<span style="color: #1e009c; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">Interactive Visualizations with Wandb</span>

Let's use Weights & Biases Log Tables and Logging to interactively visualize our voxels. 

In [ ]:
!pip install wandb --upgrade -q

## Supress wandb logs since we are not going to train in this notebook
os.environ["WANDB_SILENT"] = "true"

import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
key = user_secrets.get_secret("wandb_key")
wandb.login(key=key)

In [ ]:
dataset = '/kaggle/input/rsna-processed-voxels-64x256x256'
train_voxels = sorted(glob.glob(f"{dataset}/voxels/*/*.npy"))

df_train = pd.DataFrame(train_voxels, columns=['voxel_paths'])
df_train['BraTS21ID'] = df_train.voxel_paths.map(lambda path:path.split('/')[-1].strip('.npy'))
df_train['scan_type'] = df_train.voxel_paths.map(lambda path:path.split('/')[-2])

df_train_labels = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv',
                             dtype={'BraTS21ID':np.object,
                                   'MGMT_value':np.int32})
df_train = df_train.set_index('BraTS21ID').join(df_train_labels.set_index('BraTS21ID'), on='BraTS21ID', how='left')
df_train = df_train.reset_index()
df_train.to_csv("/kaggle/working/train_df_meta.csv")
df_train.head(3)

In [ ]:
run = wandb.init(project='RSNA-public-viz', job_type='augmentations-viz',
                name='samples')
os.makedirs('gifs/', exist_ok=True)

for i, path in tqdm(enumerate(train_voxels[:16]), total=16):
    voxel = np.load(path)
    gifs = np.expand_dims(voxel, axis=-1).astype('uint8')
    imageio.mimsave(f'gifs/voxel_{i}.gif', gifs)    

wandb.log({'images': [wandb.Image(f'gifs/voxel_{i}.gif') for i in range(16)]})
run.finish()

In [ ]:
display(HTML(f'<a href={run.get_url()} style="font-family: Segoe UI;font-size: 1.5em; font-weight: 300;">View the Full Dashboard Here 🔎</a>'))
print()
display(HTML(f"<div style='width: 780px; height: 500px; padding: 0; overflow: hidden;'><iframe src='{run.get_url()}', width=1010, height=650, style='-ms-zoom: 0.75; -moz-transform: scale(0.75); -moz-transform-origin: 0 0; -o-transform: scale(0.75); -o-transform-origin: 0 0; -webkit-transform: scale(0.75); -webkit-transform-origin: 0 0;'></iframe></div>"))

In [ ]:
run = wandb.init(project='RSNA-public-viz', name='voxels-processed')
voxels_table = wandb.Table(columns=['patient_id', 'MGMT_value',
                                    'FLAIR', 'T1w', 'T1wCE', 'T2w'])

os.makedirs('sample_gifs/', exist_ok=True)
scan_types = ['FLAIR','T1w','T1wCE','T2w']

for patient_id in tqdm(df_train.BraTS21ID.unique()[:4]):
    voxel_paths = df_train.loc[df_train.BraTS21ID==patient_id, 'voxel_paths']
    label = df_train.loc[df_train.BraTS21ID==patient_id, 'MGMT_value'].values[0]
    for path in voxel_paths:
        voxels = np.load(path)
        scan_type = path.split('/')[-2]
        gifs = voxels.clip(0, 255).astype('uint8')
        os.makedirs(f'sample_gifs/{scan_type}', exist_ok=True)
        imageio.mimsave(f'sample_gifs/{scan_type}/{patient_id}.gif', gifs)

    voxels_table.add_row(patient_id, label,
             *[wandb.Image(f'sample_gifs/{scan_type}/{patient_id}.gif') for scan_type in scan_types])


wandb.log({'voxels_processed_dataset': voxels_table})
run.finish()

<p style='text-align: left;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.6em; font-weight: 300;"><a href="https://wandb.ai/sreevishnu-damodaran/RSNA-public-viz/runs/2s0rbc66">View the Full Dashboard Here 🔎</a></span></p>
<br>

![](https://i.ibb.co/wz3d8Tz/gif-tab12.gif)

<span style="color: #1e009c; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">Read & Use Voxels Directly in Tensorflow</span>

We can directly use the voxels without tfrecord conversion. We have to use the `tf.io.read_file` and `tf.io.decode_raw` methods in tensorflow to read the 3D voxels as there are no methods to decode '.npy' files specifically in tensorflow yet.

The final tensor is obtained by removing the header values computed in the `get_npy_header_cnt` function below.

In [ ]:
dataset = '/kaggle/input/rsna-processed-voxels-64x256x256'
input_dims = (64, 256, 256)
voxel_dtype = tf.uint8

def get_npy_header_cnt(sample_voxel, dtype=tf.uint8):
    img = np.load(sample_voxel)
    tf_img = tf.io.read_file(sample_voxel)
    tf_img = tf.io.decode_raw(tf_img, tf.uint8)
    return tf_img.shape[0]-int(np.prod(img.shape))

# Some sample voxel
npy_header_size = get_npy_header_cnt(f'{dataset}/voxels/FLAIR/00830.npy',
                                     voxel_dtype)

In [ ]:
train_voxels = sorted(glob.glob(f"{dataset}/voxels/*/*.npy"))

df_train = pd.DataFrame(train_voxels, columns=['voxel_paths'])
df_train['BraTS21ID'] = df_train.voxel_paths.map(lambda path:path.split('/')[-1].strip('.npy'))
df_train['scan_type'] = df_train.voxel_paths.map(lambda path:path.split('/')[-2])

for cnt, path in enumerate(df_train.voxel_paths[:32]):
    voxel = tf.io.read_file(path)
    voxel = tf.io.decode_raw(voxel, voxel_dtype)
    voxel = voxel[npy_header_size:]
    voxel = tf.cast(voxel, tf.float32)/255.0
    voxel = tf.reshape(voxel, input_dims)
    voxel = tf.expand_dims(voxel, axis=-1)
    if cnt==0:
        voxels = tf.expand_dims(voxel, axis=0)
    else:
        voxels = tf.concat([voxels, tf.expand_dims(voxel, axis=0)], axis=0)

<span style="color: #1e009c; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">TPU Augmentations</span>

- Maximum and minimum values or delta values can be specifed for most of the augmenations below and the parameters for the operation are randomly choosen from this range for each voxel.

- Even though, the parameters are picked randomly, we ensure the same augmentation is applied across all frames in a voxel to preserve the planar alignment and to preserve the image characteristics across frames.

- There is also a provision added to specify the probability for which each augmenation gets applied to the voxels from each batch.

- Since we use the voxel size 64x256x256 in the current version of this notebook, the augmentations that does geometrical transformations are applied with to the Y and the Z axis i.e to the axial MRI slices. This is done so because, the XY and the XZ planes are small (64x256) and they have very less information. For instance, applying the cutout operations to the XY and the XZ planes would drastically reduce the little information by creating holes. The same goes for random crop transformations. It would also not be possible to apply rotations like yaw and pitch as the XY and XZ planes are small compared to the YZ plane (256, 256) so we just apply roll (YZ rotation).

- To perform some of the transformations below, I have used some functions for image processing from `tf.image` and `tfa.image` modules. Some of those functions inherently transform 4D images (3D with channel), as their designated inputs are often image batches. This property is carefully used in some places to apply transformations to the 3D voxel as a whole.

<br>

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">Light Augmentations</span>

In [ ]:
FLIP = 1.0 # @params: probability
CONTRAST = (0.5, 1.5, 1.0) # @params: (minval, maxval, probability)
BRIGHTNESS = (0.1, 1.0) # @params: (delta, probability)
GAMMA = (0.5, 1.5, 1.0) # @params: (minval, maxval, probability)

seed = 53

def augment(voxel):
    aug_seed = tf.random.uniform((2,), minval=1, maxval=9999, dtype=tf.int32)
    if tf.random.uniform(()) < FLIP:
        if tf.random.uniform(()) < 0.5:
            voxel = tf.image.flip_up_down(voxel)
        else:
            voxel = tf.image.flip_left_right(voxel)

    if tf.random.uniform(()) < BRIGHTNESS[1]:
        voxel = tf.image.adjust_brightness(voxel, tf.random.uniform((), minval=0.0,
                                                                maxval=BRIGHTNESS[0],
                                                                seed=seed))
    if tf.random.uniform(()) < CONTRAST[2]:
        voxel = tf.image.adjust_contrast(voxel, tf.random.uniform((), minval=CONTRAST[0],
                                                              maxval=CONTRAST[1],
                                                              seed=seed))
    if tf.random.uniform(()) < GAMMA[2]:
        voxel = tf.image.adjust_gamma(voxel, tf.random.uniform((), minval=GAMMA[0],
                                                           maxval=GAMMA[1],
                                                           seed=seed))
    voxel = tf.where(tf.math.is_nan(voxel), tf.zeros_like(voxel), voxel)
    return voxel

os.makedirs('gifs/', exist_ok=True)
run = wandb.init(project='RSNA-public-viz', job_type='augmentations-viz',
                name='light-augmentations')

for i, voxel in tqdm(enumerate(voxels), total=voxels.shape[0]):
    gifs = (augment(voxel)*255.).numpy().clip(0, 255).astype('uint8')
    imageio.mimsave(f'gifs/voxel_{i}.gif', gifs)  

wandb.log({'images': [wandb.Image(f'gifs/voxel_{i}.gif') for i in range(voxels.shape[0])]})
run.finish()

In [ ]:
display(HTML(f'<a href={run.get_url()} style="font-family: Segoe UI;font-size: 1.5em; font-weight: 300;">View the Full Dashboard Here 🔎</a>'))
print()
display(HTML(f"<div style='width: 780px; height: 500px; padding: 0; overflow: hidden;'><iframe src='{run.get_url()}', width=1010, height=650, style='-ms-zoom: 0.75; -moz-transform: scale(0.75); -moz-transform-origin: 0 0; -o-transform: scale(0.75); -o-transform-origin: 0 0; -webkit-transform: scale(0.75); -webkit-transform-origin: 0 0;'></iframe></div>"))

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">Random Cutout</span>

In [ ]:
CUTOUT = ((10, 10), 20, 1.0) # @params: ((mask_dim0, mask_dim1), max_num_holes, probability)

def random_cutout3D(voxel, mask_shape=(10, 10), num_holes=20, p=0.5):
    if tf.random.uniform(()) < p:
        voxel_shape = voxel.shape
        assert voxel_shape[1] >= mask_shape[0]
        assert voxel_shape[2] >= mask_shape[1]

        holes = tf.random.uniform((), minval=1, maxval=num_holes,
                                  dtype=tf.int32)
        mask_size = tf.constant([mask_shape[0], mask_shape[1]])
        mask = tf.Variable((lambda : tf.ones(voxel_shape)),
                           trainable=False)
        
        for i in tf.range(holes):
            x = tf.random.uniform((), minval=0, maxval=voxel_shape[2],
                                      dtype=tf.int32)
            y = tf.random.uniform((), minval=0, maxval=voxel_shape[1],
                                      dtype=tf.int32)
            mask_endx = tf.add(x, mask_size[1])
            mask_endy = tf.add(y, mask_size[0])
            mask[:, x:mask_endx,
                 y:mask_endy, :].assign(tf.zeros_like(mask[:, x:mask_endx,
                                                        y:mask_endy, :]))
        voxel = tf.multiply(voxel, mask)
        mask.assign(tf.ones(voxel_shape))
    return voxel

def augment(voxel):
    voxel = random_cutout3D(voxel, mask_shape=CUTOUT[0],
                          num_holes=CUTOUT[1], p=CUTOUT[2])
    voxel = tf.where(tf.math.is_nan(voxel), tf.zeros_like(voxel), voxel)
    return voxel

run = wandb.init(project='RSNA-public-viz', job_type='augmentations-viz',
                name='cutout-augmentations')
os.makedirs('gifs/', exist_ok=True)

for i, voxel in tqdm(enumerate(voxels), total=voxels.shape[0]):
    gifs = (augment(voxel)*255.).numpy().clip(0, 255).astype('uint8')
    imageio.mimsave(f'gifs/voxel_{i}.gif', gifs)    

wandb.log({'images': [wandb.Image(f'gifs/voxel_{i}.gif') for i in range(voxels.shape[0])]})
run.finish()

In [ ]:
display(HTML(f'<a href={run.get_url()} style="font-family: Segoe UI;font-size: 1.5em; font-weight: 300;">View the Full Dashboard Here 🔎</a>'))
print()
display(HTML(f"<div style='width: 780px; height: 500px; padding: 0; overflow: hidden;'><iframe src='{run.get_url()}', width=1010, height=650, style='-ms-zoom: 0.75; -moz-transform: scale(0.75); -moz-transform-origin: 0 0; -o-transform: scale(0.75); -o-transform-origin: 0 0; -webkit-transform: scale(0.75); -webkit-transform-origin: 0 0;'></iframe></div>"))

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">Random Rotate</span>

In [ ]:
ROTATE = (20, 1.0) # @params: (maxangle, probability)

def random_rotate3D(voxel, limit=90, p=0.5):
    if tf.random.uniform(()) < p:
        angle = tf.random.uniform((), minval=-limit, maxval=limit, dtype=tf.int32)
        voxel = tfa.image.rotate(voxel, tf.cast(angle, tf.float32)*(math.pi/180),
                               interpolation='nearest', fill_mode='constant',
                                 fill_value=0.0)
    return voxel

def augment(voxel):
    voxel = random_rotate3D(voxel, limit=ROTATE[0], p=ROTATE[1])
    voxel = tf.where(tf.math.is_nan(voxel), tf.zeros_like(voxel), voxel)
    return voxel

run = wandb.init(project='RSNA-public-viz', job_type='augmentations-viz',
                name='rotate-augmentations')
os.makedirs('gifs/', exist_ok=True)

for i, voxel in tqdm(enumerate(voxels), total=voxels.shape[0]):
    gifs = (augment(voxel)*255.).numpy().clip(0, 255).astype('uint8')
    imageio.mimsave(f'gifs/voxel_{i}.gif', gifs)    

wandb.log({'images': [wandb.Image(f'gifs/voxel_{i}.gif') for i in range(voxels.shape[0])]})
run.finish()

In [ ]:
display(HTML(f'<a href={run.get_url()} style="font-family: Segoe UI;font-size: 1.5em; font-weight: 300;">View the Full Dashboard Here 🔎</a>'))
print()
display(HTML(f"<div style='width: 780px; height: 500px; padding: 0; overflow: hidden;'><iframe src='{run.get_url()}', width=1010, height=650, style='-ms-zoom: 0.75; -moz-transform: scale(0.75); -moz-transform-origin: 0 0; -o-transform: scale(0.75); -o-transform-origin: 0 0; -webkit-transform: scale(0.75); -webkit-transform-origin: 0 0;'></iframe></div>"))

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">Random Resized Crop</span>

In [ ]:
RANDOM_CROP = (180, 180, 1.0) # @params: (min_width, min_height, probability)

def random_resized_crop3D(voxel, min_width, min_height, p=0.5):
    if tf.random.uniform(()) < p:
        voxel_shape = voxel.shape
        assert voxel_shape[1] >= min_height
        assert voxel_shape[2] >= min_width
        
        width = tf.random.uniform((), minval=min_width, maxval=voxel_shape[2],
                                  dtype=tf.int32)
        height = tf.random.uniform((), minval=min_height, maxval=voxel_shape[1],
                                   dtype=tf.int32)
        x = tf.random.uniform((), minval=0, maxval=voxel_shape[2] - width,
                                  dtype=tf.int32)
        y = tf.random.uniform((), minval=0, maxval=voxel_shape[1] - height,
                                  dtype=tf.int32)
        voxel = voxel[:, y:y+height, x:x+width, :]
        voxel = tf.image.resize(voxel, voxel_shape[1:3], method='lanczos5')
    return voxel

def augment(voxel):
    voxel = random_resized_crop3D(voxel, RANDOM_CROP[0], RANDOM_CROP[1],
                              p=RANDOM_CROP[2])
    voxel = tf.where(tf.math.is_nan(voxel), tf.zeros_like(voxel), voxel)
    return voxel


run = wandb.init(project='RSNA-public-viz', job_type='augmentations-viz',
                name='random-crop-augmentations')
os.makedirs('gifs/', exist_ok=True)

for i, voxel in tqdm(enumerate(voxels), total=voxels.shape[0]):
    gifs = (augment(voxel)*255.).numpy().clip(0, 255).astype('uint8')
    imageio.mimsave(f'gifs/voxel_{i}.gif', gifs)    

wandb.log({'images': [wandb.Image(f'gifs/voxel_{i}.gif') for i in range(voxels.shape[0])]})
run.finish()

In [ ]:
display(HTML(f'<a href={run.get_url()} style="font-family: Segoe UI;font-size: 1.5em; font-weight: 300;">View the Full Dashboard Here 🔎</a>'))
print()
display(HTML(f"<div style='width: 780px; height: 500px; padding: 0; overflow: hidden;'><iframe src='{run.get_url()}', width=1010, height=650, style='-ms-zoom: 0.75; -moz-transform: scale(0.75); -moz-transform-origin: 0 0; -o-transform: scale(0.75); -o-transform-origin: 0 0; -webkit-transform: scale(0.75); -webkit-transform-origin: 0 0;'></iframe></div>"))

<span style="color: #111111; font-family: Segoe UI; font-size: 1.7em; font-weight: 300;">Random Blur</span>

In [ ]:
from tensorflow_addons.image import utils as img_utils

BLUR = ([3, 3], 50, 1.0) # @params: ((filter_dim0, filter_dim1), max_sigma, probability)

def _get_gaussian_kernel(sigma, filter_shape):
    """Compute 1D Gaussian kernel."""
    x = tf.range(-filter_shape // 2 + 1, filter_shape // 2 + 1)
    x = tf.cast(x ** 2, sigma.dtype)
    x = tf.nn.softmax(-x / (2.0 * (sigma ** 2)))
    return x

def random_gaussian_blur3D(voxel, filter_shape=[5, 5], max_sigma=3, p=0.5):
    if tf.random.uniform(()) < p:
        sigma = tf.random.uniform((), minval=3, maxval=max_sigma,
                          dtype=tf.int32)
        filter_shape = tf.constant(filter_shape)
        channels = voxel.shape[-1]
        sigma = tf.cast(sigma, voxel.dtype)
        gaussian_kernel_x = _get_gaussian_kernel(sigma, filter_shape[1])
        gaussian_kernel_x = gaussian_kernel_x[tf.newaxis, :]
        gaussian_kernel_y = _get_gaussian_kernel(sigma, filter_shape[0])        
        gaussian_kernel_y = gaussian_kernel_y[:, tf.newaxis]
        gaussian_kernel_2d = tf.matmul(gaussian_kernel_y, gaussian_kernel_x)
        gaussian_kernel_2d = gaussian_kernel_2d[:, :, tf.newaxis, tf.newaxis]
        gaussian_kernel_2d = tf.tile(gaussian_kernel_2d,
                                     tf.constant([1, 1, channels, 1]))
        voxel = tf.nn.depthwise_conv2d(input=voxel,
                                       filter=gaussian_kernel_2d,
                                       strides=(1, 1, 1, 1),
                                       padding="SAME",
                                       )
        voxel = tf.cast(voxel, voxel.dtype)
    return voxel

def augment(voxel):
    voxel = random_gaussian_blur3D(voxel, filter_shape=BLUR[0],
                                   max_sigma=BLUR[1], p=BLUR[2])
    voxel = tf.where(tf.math.is_nan(voxel),
                     tf.zeros_like(voxel), voxel)
    return voxel

run = wandb.init(project='RSNA-public-viz', job_type='augmentations-viz',
                name='blur-augmentations')
os.makedirs('gifs/', exist_ok=True)

for i, voxel in tqdm(enumerate(voxels), total=voxels.shape[0]):
    gifs = (augment(voxel)*255.).numpy().clip(0, 255).astype('uint8')
    imageio.mimsave(f'gifs/voxel_{i}.gif', gifs)    

wandb.log({'images': [wandb.Image(f'gifs/voxel_{i}.gif') for i in range(voxels.shape[0])]})
run.finish()

In [ ]:
display(HTML(f'<a href={run.get_url()} style="font-family: Segoe UI;font-size: 1.5em; font-weight: 300;">View the Full Dashboard Here 🔎</a>'))
print()
display(HTML(f"<div style='width: 780px; height: 500px; padding: 0; overflow: hidden;'><iframe src='{run.get_url()}', width=1010, height=650, style='-ms-zoom: 0.75; -moz-transform: scale(0.75); -moz-transform-origin: 0 0; -o-transform: scale(0.75); -o-transform-origin: 0 0; -webkit-transform: scale(0.75); -webkit-transform-origin: 0 0;'></iframe></div>"))

<span style="color: #1e009c; font-family: Segoe UI; font-size: 1.8em; font-weight: 300;">🛰️ Putting It All Together</span>

In [ ]:
seed = 42
dataset = '/kaggle/input/rsna-processed-voxels-64x256x256-clahe'
input_dims = (64, 256, 256)
voxel_dtype = tf.uint8

FLIP = 0.5 # @params: probability
CONTRAST = (0.7, 1.3, 0.4) # @params: (minval, maxval, probability)
BRIGHTNESS = (0.2, 0.4) # @params: (delta, probability)
GAMMA = (0.8, 1.2, 0.4) # @params: (minval, maxval, probability)
ROTATE = (20, 0.3) # @params: (maxangle, probability)
RANDOM_CROP = (180, 180, 0.3) # @params: (min_width, min_height, probability)
CUTOUT = ((12, 12), 15, 0.3) # @params: ((mask_dim0, mask_dim1), max_num_holes, probability)
BLUR = ([3, 3], 50, 0.4) # @params: ((filter_dim0, filter_dim1), max_sigma, probability)

def build_decoder(with_labels=True, target_size=(64, 256, 256), ext='npy'):
    def decode(path):
        if ext == 'npy':
            voxel = tf.io.read_file(path)
            voxel = tf.io.decode_raw(voxel, voxel_dtype)
            voxel = voxel[npy_header_size:]
        else:
            raise ValueError("voxel extension not supported")
        
        voxel = tf.cast(voxel, tf.float32)/255.0
        voxel = tf.reshape(voxel, target_size)
        voxel = tf.expand_dims(voxel, axis=-1)

        return voxel
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

def build_augmenter(with_labels=True):
    '''
    Performing stateless contrast and brightness tranformations with the same seed
    to ensure the same tranformation is applied to every voxel slice.
    ''' 
    def augment(voxel):
        aug_seed = tf.random.uniform((2,), minval=1, maxval=9999, dtype=tf.int32)
        if tf.random.uniform(()) < FLIP:
            if tf.random.uniform(()) < 0.5:
                voxel = tf.image.flip_up_down(voxel)
            else:
                voxel = tf.image.flip_left_right(voxel)
                
        if tf.random.uniform(()) < BRIGHTNESS[1]:
            voxel = tf.image.adjust_brightness(voxel, tf.random.uniform((), minval=0.0,
                                                                    maxval=BRIGHTNESS[0],
                                                                    seed=seed))
        if tf.random.uniform(()) < CONTRAST[2]:
            voxel = tf.image.adjust_contrast(voxel, tf.random.uniform((), minval=CONTRAST[0],
                                                                  maxval=CONTRAST[1],
                                                                  seed=seed))
        if tf.random.uniform(()) < GAMMA[2]:
            voxel = tf.image.adjust_gamma(voxel, tf.random.uniform((), minval=GAMMA[0],
                                                               maxval=GAMMA[1],
                                                               seed=seed))
        voxel = random_rotate3D(voxel, limit=ROTATE[0], p=ROTATE[1])
        voxel = random_resized_crop3D(voxel, RANDOM_CROP[0], RANDOM_CROP[1],
                                  p=RANDOM_CROP[2])
        voxel = random_gaussian_blur3D(voxel, filter_shape=BLUR[0],
                                       max_sigma=BLUR[1], p=BLUR[2])
        voxel = random_cutout3D(voxel, mask_shape=CUTOUT[0],
                          num_holes=CUTOUT[1], p=CUTOUT[2])
        
        voxel = tf.where(tf.math.is_nan(voxel), tf.zeros_like(voxel), voxel)
        voxel = tf.cast(voxel, tf.float32)
        return voxel
    
    def augment_with_labels(voxel, label):
        return augment(voxel), label
    
    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024,
                  seed=None, cache_dir=""):
    
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else ds
    
    ## Map the functions to perform Augmentations
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle, seed=seed) if shuffle else dset
    dset = dset.batch(bsize, drop_remainder=True).prefetch(AUTO)
    return dset

In [ ]:
label_cols = 'MGMT_value'
train_paths = df_train.voxel_paths.values
decoder = build_decoder(with_labels=False, target_size=input_dims, ext='npy')

train_dataset = build_dataset(
    train_paths, bsize=32, decode_fn=decoder,
    augment=True, repeat=False, shuffle=True
)

train_voxels = next(iter(train_dataset))

run = wandb.init(project='RSNA-public-viz', job_type='augmentations-viz',
                name='all-augmentations')
os.makedirs('gifs/', exist_ok=True)

for i, voxel in tqdm(enumerate(train_voxels), total=train_voxels.shape[0]):
    gifs = (voxel*255.).numpy().clip(0, 255).astype('uint8')
    imageio.mimsave(f'gifs/voxel_{i}.gif', gifs)    

wandb.log({'images': [wandb.Image(f'gifs/voxel_{i}.gif') for i in range(voxels.shape[0])]})
run.finish()

In [ ]:
display(HTML(f'<a href={run.get_url()} style="font-family: Segoe UI;font-size: 1.5em; font-weight: 300;">View the Full Dashboard Here 🔎</a>'))
print()
display(HTML(f"<div style='width: 780px; height: 500px; padding: 0; overflow: hidden;'><iframe src='{run.get_url()}', width=1010, height=650, style='-ms-zoom: 0.75; -moz-transform: scale(0.75); -moz-transform-origin: 0 0; -o-transform: scale(0.75); -o-transform-origin: 0 0; -webkit-transform: scale(0.75); -webkit-transform-origin: 0 0;'></iframe></div>"))

#### Training Notebook:

[[TPU] RSNA Keras 3D CNN Voxel Train 🏷️🏂](https://www.kaggle.com/sreevishnudamodaran/tpu-rsna-keras-3d-cnn-voxel-train)<br>

In [ ]:
!find ./wandb -name "*.gif" -type f -delete
!find ./sample_gifs -name "*.gif" -type f -delete
!rm -r ./voxels

<p style='text-align: center;'><span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300; letter-spacing:3px">HAVE A GREAT DAY !</span></p>

<p style='text-align: center;'><span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Let me know if you have any suggestions!</span></p>